In [1]:
# import packages
from bs4 import BeautifulSoup
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#set URL, response and soup
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# find the class of the object in wikipedia, and set soup.find to get it's body
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [4]:
#every new row includes <tr>, so find them
rows = table.find_all('tr')

In [5]:
# every column starts with <th>, identify them and set columns
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

In [6]:
#make the dataframe using Pandas
df = pd.DataFrame(columns=columns)

In [7]:
#transform rows to text and append them to the Dataframe

for i in range (1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]
    
    else:
        values = [td.text.replace('\n','') for td in tds]
        
    print(values)
    
    df=df.append(pd.Series(values, index=columns), ignore_index = True)

['M1A', 'Not assigned', 'Not assigned']
['M2A', 'Not assigned', 'Not assigned']
['M3A', 'North York', 'Parkwoods']
['M4A', 'North York', 'Victoria Village']
['M5A', 'Downtown Toronto', 'Harbourfront']
['M5A', 'Downtown Toronto', 'Regent Park']
['M6A', 'North York', 'Lawrence Heights']
['M6A', 'North York', 'Lawrence Manor']
['M7A', "Queen's Park", 'Not assigned']
['M8A', 'Not assigned', 'Not assigned']
['M9A', 'Etobicoke', 'Islington Avenue']
['M1B', 'Scarborough', 'Rouge']
['M1B', 'Scarborough', 'Malvern']
['M2B', 'Not assigned', 'Not assigned']
['M3B', 'North York', 'Don Mills North']
['M4B', 'East York', 'Woodbine Gardens']
['M4B', 'East York', 'Parkview Hill']
['M5B', 'Downtown Toronto', 'Ryerson']
['M5B', 'Downtown Toronto', 'Garden District']
['M6B', 'North York', 'Glencairn']
['M7B', 'Not assigned', 'Not assigned']
['M8B', 'Not assigned', 'Not assigned']
['M9B', 'Etobicoke', 'Cloverdale']
['M9B', 'Etobicoke', 'Islington']
['M9B', 'Etobicoke', 'Martin Grove']
['M9B', 'Etobicoke',

['M4W', 'Downtown Toronto', 'Rosedale']
['M5W', 'Downtown Toronto', 'Stn A PO Boxes 25 The Esplanade']
['M6W', 'Not assigned', 'Not assigned']
['M7W', 'Not assigned', 'Not assigned']
['M8W', 'Etobicoke', 'Alderwood']
['M8W', 'Etobicoke', 'Long Branch']
['M9W', 'Etobicoke', 'Northwest']
['M1X', 'Scarborough', 'Upper Rouge']
['M2X', 'Not assigned', 'Not assigned']
['M3X', 'Not assigned', 'Not assigned']
['M4X', 'Downtown Toronto', 'Cabbagetown']
['M4X', 'Downtown Toronto', 'St. James Town']
['M5X', 'Downtown Toronto', 'First Canadian Place']
['M5X', 'Downtown Toronto', 'Underground city']
['M6X', 'Not assigned', 'Not assigned']
['M7X', 'Not assigned', 'Not assigned']
['M8X', 'Etobicoke', 'The Kingsway']
['M8X', 'Etobicoke', 'Montgomery Road']
['M8X', 'Etobicoke', 'Old Mill North']
['M9X', 'Not assigned', 'Not assigned']
['M1Y', 'Not assigned', 'Not assigned']
['M2Y', 'Not assigned', 'Not assigned']
['M3Y', 'Not assigned', 'Not assigned']
['M4Y', 'Downtown Toronto', 'Church and Wellesley'

In [8]:
#remove duplicates

df = df.drop_duplicates()

In [9]:
#Voila! the DataFrame!!!
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [10]:
#Remove rows where Borough is not assigned and reindex

df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [11]:
# Check if there are not assigned Neighbourhoods
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [12]:
# replace not assigned Neighborhoods with Borough values
df.loc[df['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df['Borough']

In [13]:
# check again
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood


In [14]:
#group by Postcode and combine neighborhood rows

df = df.groupby(['Postcode', 'Borough']).agg(lambda col: ', '.join(col)).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [15]:
#shows the shape of the DF (rows, columns)
df.shape

(103, 3)

In [16]:
#import and see the file
df2 = pd.read_csv("C:\\Users\\Neosan\\Documents\\Courses\\Coursera\\IBM\\Geospatial_Coordinates.csv")
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [17]:
# Rename columns
df2.columns = ['Postcode', 'Latitude', 'Longitude']
df2

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [18]:
# Merge Data Frames
df_merged = pd.merge(df,
                 df2[['Postcode', 'Latitude', 'Longitude']],
                 on='Postcode')

In [19]:
#Shows the merged Dataframe
df_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [20]:
#Show results that contain York in Neighbourhood

df_merged = df_merged[df_merged['Borough'].str.contains("York")]

In [21]:
df_merged.reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [22]:
#find York coords


address = 'York, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York {}, {}.'.format(latitude, longitude))

GeocoderTimedOut: Service timed out

In [ ]:
#set coords into map


# create map of  York using latitude and longitude values
map_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

In [ ]:
#Define Foursquare credentials
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

CLIENT_ID = 'DXSPOQ2A33NUCMOJHQ5410TDDZ2T3ELGGR0C4QSGRBMLD3MB' # your Foursquare ID
CLIENT_SECRET = 'O1E1VQKXC42XRH5AFUYDGNILCKQS03WJIMLGKLETLSIMEVKQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#define a function to explore York

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:

york_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )


In [ ]:
#Check size of the resultinf DF

print(york_venues.shape)
york_venues.head()

In [ ]:
#Let's check how many venues were returned for each neighborhood
york_venues.groupby('Neighborhood').count()

In [ ]:
#Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

In [ ]:
##  Analyze Each Neighborhood

# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head(5)

In [ ]:
#And let's examine the new dataframe size.
york_onehot.shape

In [ ]:
#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped

In [ ]:
#confirm new size
york_grouped.shape

In [ ]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
## Let's put that into a *pandas* dataframe

#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
## Cluster Neighborhoods


# set number of clusters
kclusters = 5

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
#let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = df_merged

In [ ]:
# merge york_grouped with york_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

york_merged#.head() # check the last columns!

In [ ]:
#Remove Nan values

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters